# G2Product Reference

## Prepare Environment

In [ ]:
import com.senzing.g2.engine.G2Product;
import com.senzing.g2.engine.G2ProductJNI;

### Helper class for Json Rendering

In [ ]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [ ]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [ ]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
    
    
}

In [ ]:
import java.util.UUID;
String str;
public static void RenderJSON(Object obj){
    str = obj.toString();
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

### Initialize Senzing configuration

Using environment variables and default values, create `senzingConfigJson`.
This value is used when instantiating Senzing objects.

In [ ]:
// Get variables used in constructing Senzing Engine configuration.

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

// Construct the JSON string used for Senzing Engine configuration.

String senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";
   
RenderJSON(senzingConfigJson);

### G2Product initialization

To start using Senzing G2Product, create and initialize an instance.
This should be done once per process.
###### Parameters

- **module_name:** (str) A short name given to this instance of the G2Product
  object.
- **senzing_config_json:** A JSON string containing configuration parameters.
- **verbose_logging:** A boolean which enables diagnostic logging.

Calling this function will return "0" upon success.

In [ ]:
String moduleName = "ExampleG2Product";
boolean verboseLogging = true;

G2Product g2product = new G2ProductJNI();
int return_code = g2product.init(moduleName, senzingConfigJson, verboseLogging);

if(return_code!=0)
    System.out.print(g2product.getLastException());
else
    System.out.print(return_code);

### license

Call `license()` to show information about the currently used license by the Senzing API

In [ ]:
String response = g2product.license();

RenderJSON(response)

### version

Call `version()` to get the version information of the current Senzing API being used

In [ ]:
String response = g2product.version();

RenderJSON(response)

### validateLicenseFile

call `validateLicenseFile()` to validate the licence file that is passed in as a parameter, returns 0 for success and 1 for failure

In [ ]:
StringBuffer response = new StringBuffer();

int return_code = g2product.validateLicenseFile("/notebooks/senzing-examples/senzing-jupyter-licenses/g2.lic", response);

if(return_code!=0)
    System.out.print(g2product.getLastException());
else
    System.out.println(return_code);
    System.out.print(response);

### validateLicenseStringBase64

call `validateLicenseStringBase64()` to validate the licence string that is passed in as a parameter, returns 0 for success and 1 for failure

In [ ]:
String licenseString = "AQAAADgCAAAAAAAAU2VuemluZyBJbnRlcm5hbCBHaXRIdWIAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAU2VuemluZyBJbnRlcm5hbCBHaXRIdSAtIDFrAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAADIwMjItMDYtMDgAAAAAAAAAAAAARVZBTAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAFNUQU5EQVJEAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAOgDAAAAAAAAMjAyNy0wNi0wOAAAAAAAAAAAAABNT05USExZAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAr38AAKFYXo35JIUTnmAPAKrmINg4vo5Dzeklb/wEg6MqFWCkgUaMbGuJnskv4PbYvnpDFYqrdwPc3YbBmD0fYXn5fT6UgV7oCkKmKr0UV+/uzsqVOJU+xcPquoDGjC8Egjqs29YDXzhl/nQTou1QqbltfzEM5B0Z58xKteHWl+Ycr+iy43pUZ1K1hECrd33hP16q1nOd/0zSDzraEeeeghbzZI6QPWEMEAw7HdBwYuoH5g9OW2EACrEUaDVgM/G9F39Qo8vulQawz0/tbd48bmuQPowsrbS6r1J5gzXMHu/zZICxU3DhktDTZ7AMGGm2ya8IBFcg9DIgtUiAzTR1rMMIn58gbgjIsnc05TgdpmwHonU4ZPYJr31ijlh1rpy25N2xSStOkqLJJwSYHiIfwP1emQ3edSXI03H50WHTtAwYIYq9ccf1cffAhdnpi0nXAFzClrskcYEb8RTBD7WV0jZwijkPq3dFnZrCX881I6RnCnux/ZHKoNrB1xnsa8KMFQw2v8An3oWWRw7VVO/8LElPVBmw0X7zpQs5wzv0u/7quM0mJvY5ouyoEhJC/t4oxgTrs54JG//v3ZZsAh47MTJYmxxEKWfG2CaaCIeLnG1oKEob8CIkJHTkyKolUjWdDxcZ+twqQrackmXB16VG/7rwnSupIRfuMKkG5kbAsNJNfx36";

StringBuffer response = new StringBuffer();

int return_code = g2product.validateLicenseStringBase64(licenseString, response);

if(return_code!=0)
    System.out.print(g2product.getLastException());
else
    System.out.println(return_code);
    System.out.print(response);

## Cleanup

In [ ]:
g2product.destroy()